# 5. Classification Model 

Data prorcessing

In [9]:

import os
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("Combined/FullCleaned.csv")

# Create a new folder to save the CSV files
folder_name = 'Indv'
os.makedirs(folder_name, exist_ok=True)

# Iterate over each unique stock
for stock_name in df['Stock'].unique():
    # Filter rows where the Stock column is equal to the current stock
    stock_df = df[df['Stock'] == stock_name]
    
    # Define the filename for the current stock
    filename = os.path.join(folder_name, f'{stock_name}.csv')
    
    # Save the filtered DataFrame to a CSV file
    stock_df.to_csv(filename, index=False)

print("done")

done


In order to have the 1 day before price we need to divide the combined dataset into individuals companies then add the price from historical prices dataset

In [16]:
import os
import pandas as pd

# Define the directories
indv_dir = 'Indv'
cleaned_prices_dir = 'CleanedHisPrices'
output_dir = 'ClassDatasets'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of files in each directory
indv_files = os.listdir(indv_dir)
cleaned_prices_files = os.listdir(cleaned_prices_dir)

# Helper function to find the most recent previous price
def get_previous_price(date, prices_df):
    previous_dates = prices_df[prices_df['Date'] < date]
    if not previous_dates.empty:
        return previous_dates.iloc[-1]['Price']
    return prices_df.iloc[-1]['Price']

# Process each file
for file_name in indv_files:
    if file_name in cleaned_prices_files:
        # Read the individual dataset
        indv_df = pd.read_csv(os.path.join(indv_dir, file_name))

        # Read the cleaned prices dataset
        cleaned_prices_df = pd.read_csv(os.path.join(cleaned_prices_dir, file_name))

        # Convert 'Date' columns to datetime
        indv_df['Date'] = pd.to_datetime(indv_df['Date'])
        cleaned_prices_df['Date'] = pd.to_datetime(cleaned_prices_df['Date'])

        # Sort the cleaned prices dataframe by date
        cleaned_prices_df = cleaned_prices_df.sort_values(by='Date').reset_index(drop=True)

        # Initialize a new column for the previous day prices
        indv_df['PreviousDayPrice'] = indv_df['Date'].apply(lambda date: get_previous_price(date, cleaned_prices_df))

        # Save the merged dataset to the output directory
        output_path = os.path.join(output_dir, file_name)
        indv_df.to_csv(output_path, index=False)

        print(f"Processed and saved: {file_name}")

print("All files processed.")


Processed and saved: 1211.csv
Processed and saved: 175.csv
Processed and saved: 2333.csv
Processed and saved: 5380.csv
Processed and saved: 7201.csv
Processed and saved: 7202.csv
Processed and saved: 7203.csv
Processed and saved: 7261.csv
Processed and saved: 7267.csv
Processed and saved: 7269.csv
Processed and saved: 7270.csv
Processed and saved: BMW.csv
Processed and saved: Ford.csv
Processed and saved: GM.csv
Processed and saved: LCID.csv
Processed and saved: LI.csv
Processed and saved: MBG.csv
Processed and saved: PII.csv
Processed and saved: PSNY.csv
Processed and saved: RACE.csv
Processed and saved: RIVN.csv
Processed and saved: RNO.csv
Processed and saved: STLAM.csv
Processed and saved: TESLA.csv
Processed and saved: VFS.csv
Processed and saved: VOLVO.csv
Processed and saved: vw.csv
All files processed.


Now after adding most recent previous day price we need to create new column for 1 increasing or 0 decreasing in price

In [22]:
import os
import pandas as pd

# Define the input and output directories
input_dir = 'ClassDatasets'
output_dir = 'ClassDatasets'


# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of files in the input directory
input_files = os.listdir(input_dir)

# Process each file
for file_name in input_files:
    file_path = os.path.join(input_dir, file_name)
    if file_path.endswith('.csv'):
        # Read the dataset
        df = pd.read_csv(file_path)

        # Check if required columns exist
        if 'Price' in df.columns and 'PreviousDayPrice' in df.columns:
            # Create the 'Increase' column
            df['Increase'] = (df['Price'] > df['PreviousDayPrice']).astype(int)

            # Drop the 'Price' column
            df.drop(columns=['Price'], inplace=True)

            # Rename the 'Increase' column to 'Price'
            df.rename(columns={'Increase': 'Price'}, inplace=True)

            # Save the processed dataset to the output directory
            output_path = os.path.join(output_dir, file_name)
            df.to_csv(output_path, index=False)

            print(f"Processed and saved: {file_name}")
        else:
            print(f"Skipping {file_name}: Required columns not found.")

print("All files processed.")



Processed and saved: 1211.csv
Processed and saved: 175.csv
Processed and saved: 2333.csv
Processed and saved: 5380.csv
Processed and saved: 7201.csv
Processed and saved: 7202.csv
Processed and saved: 7203.csv
Processed and saved: 7261.csv
Processed and saved: 7267.csv
Processed and saved: 7269.csv
Processed and saved: 7270.csv
Processed and saved: BMW.csv
Processed and saved: Ford.csv
Processed and saved: GM.csv
Processed and saved: LCID.csv
Processed and saved: LI.csv
Processed and saved: MBG.csv
Processed and saved: PII.csv
Processed and saved: PSNY.csv
Processed and saved: RACE.csv
Processed and saved: RIVN.csv
Processed and saved: RNO.csv
Processed and saved: STLAM.csv
Processed and saved: TESLA.csv
Processed and saved: VFS.csv
Processed and saved: VOLVO.csv
Processed and saved: vw.csv
All files processed.


In [24]:
import os
import pandas as pd

def combine_csv_files(folder_path, output_file):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print("Folder path does not exist.")
        return
    
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    # Check if there are CSV files in the folder
    if len(csv_files) == 0:
        print("No CSV files found in the folder.")
        return
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    # Iterate through each CSV file and append its data to the list
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dfs.append(df)
    
    # Concatenate all DataFrames in the list into one
    combined_data = pd.concat(dfs, ignore_index=True)
    
    # Write the combined data to a new CSV file
    combined_data.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}.")

# Example usage
folder_path = "ClassDatasets"
output_file = "Combined/classification_combined_data.csv"
combine_csv_files(folder_path, output_file)
print("done")


Combined data saved to Combined/classification_combined_data.csv.
done


now all combined into 1 csv we need to normalize the features

In [26]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the CSV file into a DataFrame
df = pd.read_csv('Combined/classification_combined_data.csv')

# Identify the numerical columns excluding "Price"
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop('Price')

# Standardize all numerical columns excluding "Price"
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Specify the directory to save the file
output_directory = 'Full'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Save the standardized DataFrame to a new CSV file
output_file = os.path.join(output_directory, 'Classification_Standardized_Dataset.csv')
df.to_csv(output_file, index=False)

print("Standardization complete. The standardized data has been saved to 'Standardized_Dataset.csv' in the '{}' directory.".format(output_directory))


Standardization complete. The standardized data has been saved to 'Standardized_Dataset.csv' in the 'Full' directory.


# ML

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from category_encoders import TargetEncoder

# Read the CSV file
data = pd.read_csv("Full/Classification_Standardized_Dataset.csv")

def preprocess_data(df):
    # Convert 'Date' column to datetime and extract useful features
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Quarter'] = df['Date'].dt.quarter

    # Drop original 'Date' and 'Quarter_Date' columns
    df.drop(columns=['Date'], inplace=True)
    
    # Use target encoding for the 'Stock' column
    encoder = TargetEncoder()
    df['Stock_Encoded'] = encoder.fit_transform(df['Stock'], df['Price'])
    
    # Drop the original 'Stock' column
    df.drop(columns=['Stock'], inplace=True)
    
    return df

data = preprocess_data(data)

# Assuming 'Price' is the target feature
X = data.drop(['Price'], axis=1)
y = data['Price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)
print("Random Forest:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)

# Support Vector Machine
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred)
svm_recall = recall_score(y_test, svm_pred)
print("SVM:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)

# XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred)
xgb_recall = recall_score(y_test, xgb_pred)
print("XGBoost:")
print("Accuracy:", xgb_accuracy)
print("Precision:", xgb_precision)
print("Recall:", xgb_recall)


# Neural Network
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)
nn_pred_proba = nn_model.predict(X_test_scaled)
nn_pred = np.round(nn_pred_proba).flatten()
nn_accuracy = accuracy_score(y_test, nn_pred)
nn_precision = precision_score(y_test, nn_pred)
nn_recall = recall_score(y_test, nn_pred)
print("Neural Network:")
print("Accuracy:", nn_accuracy)
print("Precision:", nn_precision)
print("Recall:", nn_recall)



Random Forest:
Accuracy: 0.5124378109452736
Precision: 0.5471698113207547
Recall: 0.5370370370370371
SVM:
Accuracy: 0.5970149253731343
Precision: 0.6030534351145038
Recall: 0.7314814814814815
XGBoost:
Accuracy: 0.5124378109452736
Precision: 0.5462962962962963
Recall: 0.5462962962962963


D:\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  
Neural Network:
Accuracy: 0.6019900497512438
Precision: 0.6044776119402985
Recall: 0.75


## Hyperparameters

In [29]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Kernel coefficient
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # Kernel type
}

# Create the GridSearchCV object
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_

# Make predictions with the best model
best_svm_pred = best_svm_model.predict(X_test_scaled)

# Evaluate the best model
best_svm_accuracy = accuracy_score(y_test, best_svm_pred)
best_svm_precision = precision_score(y_test, best_svm_pred)
best_svm_recall = recall_score(y_test, best_svm_pred)

print("Best SVM Parameters:")
print(best_params)
print("Best SVM Model:")
print(best_svm_model)
print("Best SVM Performance:")
print("Accuracy:", best_svm_accuracy)
print("Precision:", best_svm_precision)
print("Recall:", best_svm_recall)


Best SVM Parameters:
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Best SVM Model:
SVC(C=1, gamma=0.1)
Best SVM Performance:
Accuracy: 0.5920398009950248
Precision: 0.6083333333333333
Recall: 0.6759259259259259


In [30]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object for Random Forest
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the training data
rf_grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and the best model
best_rf_params = rf_grid_search.best_params_
best_rf_model = rf_grid_search.best_estimator_

# Make predictions with the best model
best_rf_pred = best_rf_model.predict(X_test_scaled)

# Evaluate the best model
best_rf_accuracy = accuracy_score(y_test, best_rf_pred)
best_rf_precision = precision_score(y_test, best_rf_pred)
best_rf_recall = recall_score(y_test, best_rf_pred)

print("Best Random Forest Parameters:")
print(best_rf_params)
print("Best Random Forest Model:")
print(best_rf_model)
print("Best Random Forest Performance:")
print("Accuracy:", best_rf_accuracy)
print("Precision:", best_rf_precision)
print("Recall:", best_rf_recall)


D:\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
478 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "D:\Python\Python311\Lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "D:\Python\Python311\Lib\site-packages\sklearn\utils\_param_

Best Random Forest Parameters:
{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Random Forest Model:
RandomForestClassifier(max_depth=20, n_estimators=200)
Best Random Forest Performance:
Accuracy: 0.5074626865671642
Precision: 0.5436893203883495
Recall: 0.5185185185185185


In [31]:
from skopt import BayesSearchCV
from xgboost import XGBClassifier

# Define the parameter search space for XGBoost
xgb_param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (100, 1000),
    'max_depth': (3, 10),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1.0, 'uniform'),
    'colsample_bytree': (0.5, 1.0, 'uniform'),
    'gamma': (0.0, 5.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0)
}

# Create the BayesSearchCV object for XGBoost
xgb_bayes_search = BayesSearchCV(
    XGBClassifier(),
    xgb_param_space,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

# Fit the bayesian search to the training data
xgb_bayes_search.fit(X_train_scaled, y_train)

# Get the best parameters and the best model
best_xgb_params = xgb_bayes_search.best_params_
best_xgb_model = xgb_bayes_search.best_estimator_

# Make predictions with the best model
best_xgb_pred = best_xgb_model.predict(X_test_scaled)

# Evaluate the best model
best_xgb_accuracy = accuracy_score(y_test, best_xgb_pred)
best_xgb_precision = precision_score(y_test, best_xgb_pred)
best_xgb_recall = recall_score(y_test, best_xgb_pred)

print("Best XGBoost Parameters:")
print(best_xgb_params)
print("Best XGBoost Model:")
print(best_xgb_model)
print("Best XGBoost Performance:")
print("Accuracy:", best_xgb_accuracy)
print("Precision:", best_xgb_precision)
print("Recall:", best_xgb_recall)


Best XGBoost Parameters:
OrderedDict([('colsample_bytree', 0.9061979941786817), ('gamma', 0.8593578069828035), ('learning_rate', 0.1570703295827246), ('max_depth', 9), ('min_child_weight', 6), ('n_estimators', 186), ('reg_alpha', 0.7558005328358816), ('reg_lambda', 0.8726303533099421), ('subsample', 0.9559644307534418)])
Best XGBoost Model:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9061979941786817, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.8593578069828035,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1570703295827246,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=6, missing=nan, monot

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Define the function to create the neural network model
def create_nn_model(optimizer='adam', activation='relu', batch_size=32, epochs=10):
    model = Sequential([
        Dense(64, activation=activation, input_shape=(X_train_scaled.shape[1],)),
        Dense(64, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the parameter grid for Neural Network
nn_param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['relu', 'tanh'],
    'batch_size': [32, 64],
    'epochs': [10, 50]
}

best_nn_accuracy = 0
best_nn_params = {}

# Perform grid search manually
for optimizer in nn_param_grid['optimizer']:
    for activation in nn_param_grid['activation']:
        for batch_size in nn_param_grid['batch_size']:
            for epochs in nn_param_grid['epochs']:
                # Create and compile the model
                nn_model = create_nn_model(optimizer=optimizer, activation=activation, batch_size=batch_size, epochs=epochs)
                print("test")
                
                # Train the model
                nn_model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=epochs, verbose=0)
                
                # Evaluate the model
                nn_pred_proba = nn_model.predict(X_test_scaled)
                nn_pred = np.round(nn_pred_proba).flatten()
                nn_accuracy = accuracy_score(y_test, nn_pred)
                
                # Update best parameters if accuracy improves
                if nn_accuracy > best_nn_accuracy:
                    best_nn_accuracy = nn_accuracy
                    best_nn_params = {'optimizer': optimizer, 'activation': activation, 'batch_size': batch_size, 'epochs': epochs}

# Re-train the best model with the best parameters
best_nn_model = create_nn_model(**best_nn_params)
best_nn_model.fit(X_train_scaled, y_train, batch_size=best_nn_params['batch_size'], epochs=best_nn_params['epochs'], verbose=0)

# Make predictions with the best model
best_nn_pred_proba = best_nn_model.predict(X_test_scaled)
best_nn_pred = np.round(best_nn_pred_proba).flatten()

# Evaluate the best model
best_nn_accuracy = accuracy_score(y_test, best_nn_pred)
best_nn_precision = precision_score(y_test, best_nn_pred)
best_nn_recall = recall_score(y_test, best_nn_pred)

print("Best Neural Network Parameters:")
print(best_nn_params)
print("Best Neural Network Performance:")
print("Accuracy:", best_nn_accuracy)
print("Precision:", best_nn_precision)
print("Recall:", best_nn_recall)


test


D:\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
test


D:\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
test


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
test


D:\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


D:\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Best Neural Network Parameters:
{'optimizer': 'adam', 'activation': 'relu', 'batch_size': 32, 'epochs': 50}
Best Neural Network Performance:
Accuracy: 0.582089552238806
Precision: 0.59375
Recall: 0.7037037037037037
